In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_excel("data_last.xlsx")

In [2]:
X = df[['Дата', 'Модель', "Вид", "Неисправность", "Мастер", 'Сумма', 'Выдача']]
counts = X['Вид'].value_counts()
counts.tail(20)

Вид
mp3                                 14
радиола                             13
КПК                                 11
микрофон                            10
а/м dvd                              9
Блок управления климат-контролем     9
а/м кас                              9
пульт авто                           9
колонки                              8
смартфон                             5
Панель                               4
а/м cd/кас                           4
tft tv                               4
магн                                 4
тв                                   3
в/п                                  3
МП3-плеер                            2
cd dvd                               2
gps навигатор                        1
DVD-LCD                              1
Name: count, dtype: int64

In [ ]:
def get_base_cost(types, brand, model, issue):
    rules = {
        'smartphone': {
            'BrandA': {
                'ModelX': {'screen': 100, 'battery': 50},
                'ModelY': {'screen': 120, 'battery': 60}
            },
            'BrandB': {
                'ModelZ': {'screen': 90, 'battery': 45}
            }
        },
        'laptop': {
            'BrandA': {
                'Model1': {'keyboard': 150, 'screen': 200},
                'Model2': {'keyboard': 170, 'screen': 220}
            }
        },
        # Добавить правила для других типов техники
    }
    return rules.get(types, {}).get(brand, {}).get(model, {}).get(issue, None)

In [ ]:
def calculate_repair_cost(types, brand, model, issue, master, inflation_factor):
    base_cost = get_base_cost(types, brand, model, issue)
    if base_cost is None:
        return None

    master_factor = 1.1 if master == 'experienced' else 1.0  # Фактор для опытных мастеров
    adjusted_cost = base_cost * master_factor * inflation_factor
    return adjusted_cost

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Кодирование категориальных переменных
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['type', 'brand', 'model', 'issue', 'master']]).toarray()
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out())

# Объединение закодированных переменных и инфляционного показателя
final_df = pd.concat([encoded_df, df['inflation_factor']], axis=1)


In [ ]:
from sklearn.model_selection import train_test_split


X = final_df
y = df['repair_cost']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)

# Оценка модели
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'Mean Absolute Percentage Error: {mape}')

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
# Модель XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_mape = mean_absolute_percentage_error(y_test, xgb_pred)
print(f'XGBoost MAPE: {xgb_mape}')

# Модель LightGBM
lgb_model = lgb.LGBMRegressor(n_estimators=100, random_state=42)
lgb_model.fit(X_train, y_train)
lgb_pred = lgb_model.predict(X_test)
lgb_mape = mean_absolute_percentage_error(y_test, lgb_pred)
print(f'LightGBM MAPE: {lgb_mape}')

In [ ]:
# Модель CatBoost
cat_features = ['type', 'brand', 'model', 'issue', 'master']
cat_model = CatBoostRegressor(n_estimators=100, random_state=42, cat_features=cat_features, verbose=0)
cat_model.fit(df[cat_features + ['inflation_factor']], y)
cat_pred = cat_model.predict(df[cat_features + ['inflation_factor']].iloc[X_test.index])
cat_mape = mean_absolute_percentage_error(y_test, cat_pred)
print(f'CatBoost MAPE: {cat_mape}')

In [ ]:
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb


# Grid Search для случайного леса
rf_grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                              param_grid=rf_params, cv=3,
                              scoring='neg_mean_absolute_percentage_error')
rf_grid_search.fit(X_train, y_train)
rf_best_model = rf_grid_search.best_estimator_
rf_best_params = rf_grid_search.best_params_
rf_mape = mean_absolute_percentage_error(y_test, rf_best_model.predict(X_test))
print(f'Best Random Forest Params: {rf_best_params}')
print(f'Random Forest MAPE: {rf_mape}')

In [ ]:

lgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 62, 127]
}

# Grid Search для LightGBM
lgb_grid_search = GridSearchCV(estimator=lgb.LGBMRegressor(random_state=42),
                               param_grid=lgb_params, cv=3,
                               scoring='neg_mean_absolute_percentage_error')
lgb_grid_search.fit(X_train, y_train)
lgb_best_model = lgb_grid_search.best_estimator_
lgb_best_params = lgb_grid_search.best_params_
lgb_mape = mean_absolute_percentage_error(y_test, lgb_best_model.predict(X_test))
print(f'Best LightGBM Params: {lgb_best_params}')
print(f'LightGBM MAPE: {lgb_mape}')

In [ ]:

xgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Grid Search для XGBoost
xgb_grid_search = GridSearchCV(estimator=xgb.XGBRegressor(random_state=42),
                               param_grid=xgb_params, cv=3,
                               scoring='neg_mean_absolute_percentage_error')
xgb_grid_search.fit(X_train, y_train)
xgb_best_model = xgb_grid_search.best_estimator_
xgb_best_params = xgb_grid_search.best_params_
xgb_mape = mean_absolute_percentage_error(y_test, xgb_best_model.predict(X_test))
print(f'Best XGBoost Params: {xgb_best_params}')
print(f'XGBoost MAPE: {xgb_mape}')


In [17]:
X['Неисправность'].unique()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [26]:
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import numpy as np


data = {
    'issue_description': [
        'не включается', 'не тянет', 'не проигрывает', 'не читает диски',
        'не заряжается', 'не работает экран', 'плохой звук', 'не читает карты памяти'
    ]
}
df['issue_description'] = X['Неисправность']


def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

df['cleaned_issue'] = df['issue_description'].apply(clean_text)
sentences = [row.split() for row in df['cleaned_issue']]


w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Создание эмбеддингов для фраз
def get_sentence_vector(sentence):
    words = sentence.split()
    word_vectors = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(word_vectors) == 0:
        return np.zeros(100)
    return np.mean(word_vectors, axis=0)


embeddings = np.array([get_sentence_vector(sentence) for sentence in df['cleaned_issue']])


num_clusters = 20
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(embeddings)


print(df[['issue_description', 'cleaned_issue', 'cluster']])


C:\Users\leo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


                   issue_description                 cleaned_issue  cluster
0                      не включается                 не включается        0
1      не включается, заменит провод  не включается заменит провод       11
2                     не проигрывает                не проигрывает       19
3        Самопроизвольно отключается   самопроизвольно отключается       12
4                      не включается                 не включается        0
...                              ...                           ...      ...
19316                  не включается                 не включается        0
19317                 сломан штеккер                сломан штеккер       11
19318                       не греет                      не греет        0
19319                    не работает                   не работает        8
19320                    не идет пар                   не идет пар       19

[19321 rows x 3 columns]


In [27]:
df.head(30)

,Unnamed: 0,Номер,Дата,Клиент,Модель,Вид,IMEI / SN,Неисправность,Комментарии,Комментарии наши,...,Детали,Оплата в кассу,Выдача,План,Комплектация,Внешний вид,Тип,issue_description,cleaned_issue,cluster
0,0,16600,2012-04-05,Кириллова Н.С.,General satellite VA-7200,NaN,421245,не включается,1500-1800,"знает, будет 06.04",...,300,06.04.2012,06.04.2012,07.04.2012,NaN,NaN,Пл.,не включается,не включается,0
1,0,16601,2012-04-05,Артамонова Т.В.,Orion T20MS,т/в,11978,"не включается, заменит провод",1800-2100,"знает, будет 11.04",...,450,11.04.2012,11.04.2012,07.04.2012,ТВ,NaN,Пл.,"не включается, заменит провод",не включается заменит провод,11
2,0,16599,2012-04-05,Кладов Б.А.,Орфей ЭП-103C,проигр,60769,не проигрывает,"2600 ремонт механики замена иглы , согласовано","знает, будет 16.04",...,1200,17.04.2012,17.04.2012,07.04.2012,NaN,NaN,Пл.,не проигрывает,не проигрывает,19
3,0,16598,2012-04-05,Нечаев М.Ю.,Samsung TVP-3360D1X,моно TV+VHS,380514,Самопроизвольно отключается,NaN,NaN,...,0,NaN,12.04.2012,07.04.2012,NaN,NaN,Пл.,Самопроизвольно отключается,самопроизвольно отключается,12
4,0,16606,2012-04-06,Коротков А.В.,Prology DVD-515U,а/м CD,2675401,не включается,ДИАГНОСТИКА,NaN,...,0,NaN,12.04.2012,08.04.2012,база; панель,NaN,Пл.,не включается,не включается,0
5,0,16604,2012-04-06,Голубенков П.П.,Crowncorder CRC-6150S,NaN,-,не тянет,"3600 ремонт лпм, согласовано","знает, будет 23.04_x000D_\nесли притензии по з...",...,800,23.04.2012,23.04.2012,08.04.2012,NaN,NaN,Пл.,не тянет,не тянет,7
6,0,16602,2012-04-06,Зайцев В.А.,Samsung DVD-V5600,DVD+VHS,400413,полосы на экране,"2800 замена лазера, согласовано",NaN,...,0,NaN,11.05.2012,08.04.2012,DVD+VHS,NaN,Пл.,полосы на экране,полосы на экране,5
7,1,16608,2012-04-06,Золотарев Д.А.,Hitachi CS2506R,т/в,2075,не переключает программы с пульта,2500,"знает, будет 12.04",...,450,12.04.2012,12.04.2012,08.04.2012,ТВ; пульт,потертости; царапины,Пл.,не переключает программы с пульта,не переключает программы с пульта,1
8,0,16607,2012-04-06,Волкович Л.П.,JVC HR-J221A,в/м,2429,не принимает кассету,1600,"знает, будет 09.04.",...,350,09.04.2012,09.04.2012,08.04.2012,NaN,потертости; царапины,Пл.,не принимает кассету,не принимает кассету,7
9,2,16609,2012-04-06,Руденко А.С.,Sony KV-M1400K,т/в,6903,не работает,1700,"знает, будет 10.04",...,350,10.04.2012,10.04.2012,08.04.2012,ТВ,потертости; царапины,Пл.,не работает,не работает,8
